# Project 1
## 1.
Objective: $Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T} c_tx_t + \sum_{t=1}^{T} h_ts_t$

Subject to:
$d_t = x_t + s_{t-1} - s_t \qquad \forall t \in H$
$x_t \leq D_ty_t \qquad \forall t \in H$

$y_t \in [0,1]$
<br>
$x_t, s_t \geq 0$

Where:
$D_t = \sum_{i=t}^{T} d_i$
$s_0, s_T = 0$

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# Import data and define constants

T = 12  # Num periods
rep = 2  # Tab number

tabs = {'6-1': '6-periods (1)',
    '6-2': '6-periods (2)',
    '12-1': '12-periods (1)',
    '12-2': '12-periods (2)',
    '24-1': '24-periods (1)',
    '24-2': '24-periods (2)',
    '52-1': '52-periods (1)',
    '52-2': '52-periods (2)',
    '104-1': '104-periods (1)',
    '104-2': '104-periods (2)'}

xls = pd.ExcelFile(r'../ULSP-instancesR.xlsx')

df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

# Add zero row and adjust index to align with indicies prompt
df.loc[-1] = [0]*6
df.index = df.index + 1
df = df.sort_index()

# Constants
d = df['Demand Forecast']
f = df['Setup Cost']
c = df['Production cost']
h = df['Holding cost']
b = df['Backlogging cost']

D = [d[i:].sum() for i in range(len(d))]

# Set
H = range(1,T+1)
H_0 = range(T+1)

# Gurobi environment
ENV = gp.Env()
ENV.setParam('OutputFlag', 0)


Restricted license - for non-production use only - expires 2025-11-24


In [2]:
def extract_model_vars(x, y, s):
    # ---- Extract variable values ----
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))

    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X

    return x_values, y_values, s_values


def inequality_separation(x, y, s):
    S = {}
    v = {}
    Djl = {}
    for l in H:
        S[l] = set()
        for j in range(1,l+1):
            Djl[(j,l)] = d[j:l+1].sum()
            if x[j] - Djl[(j,l)]*y[j] > 0:
                S[l].add(j)
        v[l] = sum([x[t] - Djl[(t,l)]*y[t] for t in S[l]]) - s[l] # TODO: confirm, error in problem 5, summation on 3rd line should be j \in S_l, not t \in S_l?
    return v, S, Djl


def run_model(relax=False, isp=False):
    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='x')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')

    # Amount held in inventory at end of t
    s = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='s')


    # ---- Constraints ----

    # Initial values
    model.addConstr(s[0] == 0)
    model.addConstr(x[0] == 0)
    model.addConstr(y[0] == 0)

    # Material balance
    model.addConstrs(int(d[t]) == x[t] + s[t-1] - s[t] for t in H)

    # y definition
    model.addConstrs(x[t] <= D[t]*y[t] for t in H)


    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t]], [y[t], x[t], s[t]])

    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract values ----
    x_values, y_values, s_values = extract_model_vars(x, y, s)

    if isp:
        if not relax:
            print("WARNING: isp is only designed to be used when relax=True")

        v, S, Djl = inequality_separation(x_values, y_values, s_values)

        limit = 0
        count = 0
        max_count = 10000

        while max(v.values())>1e-10 and count <= max_count:
            # l_max = max(v.items(), key=lambda x: x[1])[0]
            # model.addConstrs(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l]for l in range(1,l_max+1))
            print("v: ", v)
            l = max(v.items(), key=lambda x: x[1])[0]
            print("v_max: ", max(v.items(), key=lambda x: x[1]))
            model.addConstr(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l])
            model.optimize()
            x_values, y_values, s_values = extract_model_vars(x, y, s)

            v, S, Djl = inequality_separation(x_values, y_values, s_values)

            print("final v: ", v)
            print(f"Count: {count}")

            if count == max_count-1:
                print("WARNING: isp max iterations exceeded")
            count += 1

    end_time = time.time()
    total_time = end_time - start_time

    return x_values, y_values, s_values, total_time

def print_model_results(relax, isp):
    x_values, y_values, s_values, total_time = run_model(relax=relax, isp=isp)
    print("x: ", x_values)
    print("y: ", y_values)
    print("s: ", s_values)
    print("total_time: ", total_time)
    y_list = []
    for i in range(len(y_values)):
        if y_values[i]>0:
            y_list.append(i)
    print(f"y production periods: {y_list}")

print("MILP model without relaxation:")
print_model_results(relax=False, isp=False)

print()
print("LP model with relaxation:")
print_model_results(relax=True, isp=False)

MILP model without relaxation:
x:  [  0. 623. 906.   0. 958. 319.   0. 689. 414. 372. 346.   0. 938.]
y:  [0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
s:  [ 0.  0. 71.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
total_time:  0.05498003959655762
y production periods: [1, 2, 4, 5, 7, 8, 9, 10, 12]

LP model with relaxation:
x:  [  0. 623. 835.  71. 958. 319.   0. 689. 414. 372. 346.   0. 938.]
y:  [0.         0.11194969 0.16895994 0.01728756 0.23736373 0.10363873
 0.         0.24972816 0.2        0.22463768 0.2694704  0.
 1.        ]
s:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
total_time:  0.005994558334350586
y production periods: [1, 2, 3, 4, 5, 7, 8, 9, 10, 12]


# 2.
Computation times are comparable at around 0.001s for the 6-element dataset and 0.006 for the 104-element dataset. Solutions for production quantities and holding are the same for the 6-element set, but not the 104-element set, although the y variable indicating whether a lot is produced is no longer strictly 0 or 1. If it is 0, no lot is produced, but when a lot is produced, y is a fractional value. This will not affect the production plan, but it will give an artificially lower objective.

# 3.
 Yes, we could use a heuristic here (i.e. dynamic programming) to get the same result as the MILP. In fact, this would be a good application for the Silver-Meal heuristic, which is similar to (Wagner-Whitin, 1958), except the former calculates cost per unit produced, and (Wagner-Whitin, 1958) looks at the absolute cost to fill the demand in a particular period.

Pattern: Demand for each period is filled from only one source, i.e. all demand is satisfied by production or inventory, for each period.

In [3]:
M = 1e10

production_days = set()
production_days.add(1)

costs = [M]*(T+1)

for t in H:
    # Iterate through each period

    # Period where the last production took place
    last_prod = max(production_days)

    for i in range(last_prod,t+1):
        # iterate between the last production day and period t.
        # Calculate the cost of production between the last production day and i,
        # plus the cost of production between i and t, if i were to be chosen as
        # a new production day.

        cost_after_i = f[i] + sum(d[i:t+1])*c[i] + sum([d[a+1]*sum(h[i:a+1]) for a in range(i, t)])

        if i == 1 or i <= last_prod:
            # If i is 1 or is the same as the last production period, there was no relevant demand before i
            cost_before_i = 0
        else:
            cost_before_i = f[last_prod] \
                            + sum(d[last_prod:i])*c[last_prod] \
                            + sum([d[b]*sum(list(h[last_prod:b])) for b in range(last_prod+1,i)])

        # Total cost is sum of the costs before and after i
        costs[i] =  cost_before_i + cost_after_i

    if min(costs[last_prod:]) != costs[last_prod]:
        # Look at the minimum cost between the last production and period t.
        # If this was not the last production, then add the cheapest period
        # to the set of production days
        prev_last_prod = last_prod
        production_days.add(costs.index(min(costs[last_prod:])))


# print(f"costs: {costs}, t: {t}, i: {i}")
print(f"production_days: {production_days}")
print(f"y: {[1 if i in production_days else 0 for i in H_0]}")

production_days: {1, 2, 4, 5, 7, 8, 9, 10, 12}
y: [0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]


# 4.
This inequality states that the amount of units produced in any subset of periods must be less than or equal to the demand of those periods plus the ending inventory.
Indeed, the constraint for the definition of $y_t$ is:
$x_t \leq D_ty_t \qquad \forall t \in H$

If this is the case, then cumulative (i.e. summation of) $x_t$ must also be less than the cumulative $Dy_t$. $s_l$ is defined to be nonnegative, so adding this to the right side does not change the inequality.

# 5.

See inequality_separation() function in cell 3. Example output is shown below.

In [4]:
x, y, s, _ = run_model(relax=True)
v, S, Djl = inequality_separation(x, y, s)


print(v)
print(max(v.items(), key=lambda x: x[1]))


{1: 553.2553459119497, 2: 1153.6958125577453, 3: 1203.5238128763233, 4: 1648.456465815987, 5: 1763.5517833613737, 6: 1763.5517833613737, 7: 1840.0805333664525, 8: 1803.2644266931622, 9: 1686.6180699722345, 10: 1484.8876554781589, 11: 1484.8876554781589, 12: 5.684341886080802e-14}
(7, 1840.0805333664525)


# 6.
Applying the inequality separation procedure in Gurobi.


In [5]:
print_model_results(relax=True, isp=True)



v:  {1: 553.2553459119497, 2: 1153.6958125577453, 3: 1203.5238128763233, 4: 1648.456465815987, 5: 1763.5517833613737, 6: 1763.5517833613737, 7: 1840.0805333664525, 8: 1803.2644266931622, 9: 1686.6180699722345, 10: 1484.8876554781589, 11: 1484.8876554781589, 12: 5.684341886080802e-14}
v_max:  (7, 1840.0805333664525)
final v:  {1: 225.2524353862558, 2: 693.9184540671793, 3: 751.6948820587133, 4: 1303.8753337405153, 5: 1454.6826009714366, 6: 1454.6826009714366, 7: 1608.344684309849, 8: 1617.8757474478791, 9: 1542.8746737458196, 10: 1379.8788504467125, 11: 1379.8788504467125, 12: 5.684341886080802e-14}
Count: 0
v:  {1: 225.2524353862558, 2: 693.9184540671793, 3: 751.6948820587133, 4: 1303.8753337405153, 5: 1454.6826009714366, 6: 1454.6826009714366, 7: 1608.344684309849, 8: 1617.8757474478791, 9: 1542.8746737458196, 10: 1379.8788504467125, 11: 1379.8788504467125, 12: 5.684341886080802e-14}
v_max:  (8, 1617.8757474478791)
final v:  {1: 477.311841038849, 2: 564.8503512807627, 3: 571.066328959

# 7.
New formulation:

$Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T}\sum_{q=1}^{t} c_qd_tw_{q,t} + \sum_{t=1}^T\sum_{q=1}^{t-1}\sum_{i=q}^{t-1}h_id_tw_{q,t}$
<br>
(Setup cost + unit production costs + holding cost of inventory)

Subject to:
$\sum_{q=1}^{t}w_{q,t} = 1 \qquad  \forall t \in T$     (Ensures demand is met)

$w_{q,t} \leq y_q \qquad  \forall q \in T \quad \forall t \in T$       (Defines y as 1 if any products are produced, 0 if not)
<br>
$y_t \geq 0$
<br>
$0 \leq w_{q,t} \leq 1$

Where:
$T_q = T-q$ (Remaining periods)

# 8.


In [6]:
def fplr_model(relax=False):

    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    w = model.addVars(T+1, T+1, vtype=GRB.CONTINUOUS, name='w')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')


    # ---- Constraints ----

    # Ensure demand is met
    model.addConstrs(gp.quicksum(w[q,t] for q in range(1,t+1)) == 1 for t in H)

    # Define setup variable y
    # model.addConstrs(gp.quicksum(w[q,t] for t in range(q,T+1)) <= y[q]*Tq[q] for q in H)  # Can use this constraint if y is not relaxed
    model.addConstrs(w[q,t] <= y[q] for q in H for t in range(q,T+1))


    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms(f[t], y[t])
        for q in range(1,t+1):
            obj.addTerms(c[q]*d[t], w[q,t])
        # for i in range (1,t):
        #     for q in range(1,i+1):
        #         obj.addTerms(h[t]*d[i], w[q,i])
    for t in range(1,T+1):
        for q in range(1,t):
            obj.addTerms(h[q:t].sum()*d[t], w[q,t])


    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract variable values ----
    w_values = np.zeros((len(H_0),len(H_0)))
    y_values = np.zeros(len(H_0))

    for q in H:
        for t in H:
            w_values[q,t] = w[q,t].X
        y_values[q] = y[q].X

    end_time = time.time()
    total_time = end_time - start_time

    return w_values, y_values, total_time

w_values, y_values, total_time = fplr_model(relax=True)

units_per_period = np.zeros(len(H_0))

for q in H_0:
    units_per_period[q] = np.sum([w_values[q,t]*d[t] for t in H_0])

print("w: ", w_values)
print("y: ", y_values)
print("total_time: ", total_time)
print("units_per_period: ", units_per_period)


w:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
y:  [0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
total_time:  0.004996061325073242
units_per_period:  [  0. 623. 906.   0. 958. 319.   0. 689. 414. 372. 346.   0. 938.]


# 9.
Objective: $Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T} c_tx_t + \sum_{t=1}^{T} h_ts_t + \sum_{t=1}^{T} b_tr_t$

Subject to:
$d_t = x_t + s_{t-1} - s_t - r_t + r_{t-1} \qquad \forall t \in H$
$x_t \leq Dy_t \qquad \forall t \in H$

$y_t \in [0,1]$
<br>
$x_t, s_t, r_t \geq 0$

Where:
$D = \sum_{i=0}^{T} d_i$
$s_0, s_T, r_0, r_T = 0$

In [7]:
def extract_model_vars_backlog(x, y, s, r):
    # ---- Extract variable values ----
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))
    r_values = np.zeros(len(H_0))

    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X
        r_values[t] = r[t].X

    return x_values, y_values, s_values, r_values


def run_model_backlog(relax=False, isp=False):
    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='x')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')

    # Amount held in inventory at end of t
    s = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='s')

    # Backlog at end of t
    r = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='r')

    # ---- Constraints ----

    # Initial values
    model.addConstr(s[0] == 0)
    model.addConstr(x[0] == 0)
    model.addConstr(y[0] == 0)
    model.addConstr(r[0] == 0)
    model.addConstr(r[T] == 0)

    # Material balance
    model.addConstrs(int(d[t]) == x[t] + s[t-1] - s[t] + r[t] - r[t-1] for t in H)

    # y definition
    model.addConstrs(x[t] <= D[0]*y[t] for t in H)

    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t], b[t]], [y[t], x[t], s[t], r[t]])

    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract values ----
    x_values, y_values, s_values, r_values = extract_model_vars_backlog(x, y, s, r)

    if isp:
        if not relax:
            print("WARNING: isp is only designed to be used when relax=True")

        v, S, Djl = inequality_separation(x_values, y_values, s_values)

        count = 0
        while max(v.values())>0 and count <= 100:
            # l_max = max(v.items(), key=lambda x: x[1])[0]
            # model.addConstrs(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l]for l in range(1,l_max+1))
            print("v: ", v)
            l = max(v.items(), key=lambda x: x[1])[0]
            print("l: ", l)
            print("v_max: ", max(v.items(), key=lambda x: x[1]))
            model.addConstr(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l])
            model.optimize()
            x_values, y_values, s_values = extract_model_vars(x, y, s)

            v, S, Djl = inequality_separation(x_values, y_values, s_values)

            print("final v: ", v)

            if count == 99:
                print("WARNING: isp max iterations exceeded")
            count += 1

    end_time = time.time()
    total_time = end_time - start_time

    return x_values, y_values, s_values, r_values, total_time

def print_model_results_backlog(relax, isp):
    x_values, y_values, s_values, r_values, total_time = run_model_backlog(relax=relax, isp=isp)
    print("x: ", x_values)
    print("y: ", y_values)
    print("s: ", s_values)
    print("r: ", r_values)
    print("total_time: ", total_time)

print("MILP model, with backlog, without relaxation:")
print_model_results_backlog(relax=False, isp=False)

print()
print("LP model, with backlog, with relaxation:")
print_model_results_backlog(relax=True, isp=False)

MILP model, with backlog, without relaxation:
x:  [  0. 623. 906.   0. 958. 319.   0. 689. 414.   0. 718.   0. 938.]
y:  [ 0.  1.  1.  0.  1.  1. -0.  1.  1.  0.  1. -0.  1.]
s:  [ 0.  0. 71.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
r:  [  0.   0.   0.   0.   0.   0.   0.   0.   0. 372.   0.   0.   0.]
total_time:  0.00642704963684082

LP model, with backlog, with relaxation:
x:  [  0. 623. 835.  71. 958. 319.   0. 689. 414. 372. 346.   0. 938.]
y:  [0.         0.11194969 0.15004492 0.01275831 0.17214735 0.05732255
 0.         0.12380952 0.07439353 0.06684636 0.0621743  0.
 0.16855346]
s:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
r:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
total_time:  0.0020008087158203125


# 10.
Pattern: the demand for each period is satisfied by one, and only one, source. Said another way, the demand for each period is satisfied completely by either production, inventory, or the backlog.

In [8]:
# TODO. NOT COMPLETE

M = 1e10

production_days = set()
production_days.add(1)

costs = [M]*(T+1)

for t in H:
    # Iterate through each period

    # Period where the last production took place
    last_prod = max(production_days)

    for i in range(last_prod,t+1):
        # iterate between the last production day and period t.
        # Calculate the cost of production between the last production day and i,
        # plus the cost of production between i and t, if i were to be chosen as
        # a new production day.

        cost_after_i = f[i] + sum(d[i:t+1])*c[i] + sum([d[a+1]*sum(h[i:a+1]) for a in range(i, t)])

        if i == 1 or i <= last_prod:
            # If i is 1 or is the same as the last production period, there was no relevant demand before i
            cost_before_i = 0
        else:
            cost_before_i = f[last_prod] \
                            + sum(d[last_prod:i])*c[last_prod] \
                            + sum([d[b]*sum(list(h[last_prod:b])) for b in range(last_prod+1,i)])

        # Total cost is sum of the costs before and after i
        costs[i] =  cost_before_i + cost_after_i

    if min(costs[last_prod:]) != costs[last_prod]:
        # Look at the minimum cost between the last production and period t.
        # If this was not the last production, then add the cheapest period
        # to the set of production days
        prev_last_prod = last_prod
        production_days.add(costs.index(min(costs[last_prod:])))


# print(f"costs: {costs}, t: {t}, i: {i}")
print(f"production_days: {production_days}")
print(f"y: {[1 if i in production_days else 0 for i in H_0]}")


production_days: {1, 2, 4, 5, 7, 8, 9, 10, 12}
y: [0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]


# 11.

Objective:
Min $\sum_{t \in H}w_t(d_tc_t+f_t) + \sum_{u \in H}\sum_{t=u+1}^T\phi_{ut}d_t(c_u + \sum_{i=u}^{t-1}h_i) + \sum_{u \in H}\sum_{t=1}^{u-1}\psi_{ut}d_t(c_u+\sum_{j=t}^{u-1}b_j)$
(Cost of satisfying demand in current period + cost of satisfying demand ahead of time + cost of satisfying demand via backlog)

Subject to:
<br>
$w_i \geq \phi_{it} \qquad \forall i \in H \quad \forall t \in [i+1,t]$
$w_i \geq \psi_{it} \qquad \forall i \in H \quad \forall t \in [1,i-1]$
(Production must occur in $i$ if production in this period is used to meet demand in period $t$)
<br>
$w_t + \sum_{u \in H}(\phi_{ut}+\psi_{ut}) = 1 \qquad \forall t \in H$

(demand for each period $t$ must be met)


In [14]:
def spr_model(relax=False):

    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # If demand is produced in same period
    w = model.addVars(T+1, vtype=GRB.BINARY, name='w')

    # If demand is produced beforehand
    phi = model.addVars(T+1, T+1, vtype=GRB.BINARY, name='phi')

    # If demand is produced afterwards
    psi = model.addVars(T+1, T+1, vtype=GRB.BINARY, name='psi')


    # ---- Constraints ----

    # Ensure demand is met
    model.addConstrs(w[t] + gp.quicksum(phi[u,t] + psi[u,t] for u in H) == 1 for t in H)

    # Production must occur in i if it is used to satisfy demand in t
    model.addConstrs(w[i] >= psi[i,t] for i in H for t in H)
    model.addConstrs(w[i] >= phi[i,t] for i in H for t in H)

    # Definition of phi and psi as forward- or backward-looking
    model.addConstrs(phi[u,t] == 0 if u < t else phi[u,t] >= 0 for u in H for t in H)
    model.addConstrs(psi[u,t] == 0 if u > t else psi[u,t] >= 0 for u in H for t in H)

    # Zero in period 0
    model.addConstr(w[0] == 0)
    model.addConstrs(phi[0,t] == 0 for t in H)
    model.addConstrs(psi[0,t] == 0 for t in H)


    # ---- Objective ----
    obj = gp.LinExpr()

    # Demand satisfied by production in current period
    for t in H:
        obj.addTerms(d[t]*c[t]+f[t], w[t])

    for u in H:

        # Demand satisfied by production in earlier period
        for t in range(u+1, T+1):
            obj.addTerms(
                d[t]*(sum([h[i]+c[u] for i in range(u,t)])), phi[u,t])

        # Demand satisfied by production in later period
        for t in range(1,u):
            obj.addTerms(d[t]*(sum([b[j]+c[u]for j in range(t,u)])), psi[u,t])


    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract variable values ----
    w_values = np.zeros(len(H_0))
    phi_values = np.zeros((len(H_0), len(H_0)))
    psi_values = np.zeros((len(H_0), len(H_0)))

    for u in H:
        w_values[u] = w[u].X
        for t in H:
            phi_values[u,t] = phi[u,t].X
            psi_values[u,t] = psi[u,t].X

    end_time = time.time()
    total_time = end_time - start_time

    return w_values, phi_values, psi_values, total_time

w_values, phi_values, psi_values, total_time = spr_model()

print("phi: ", phi_values)
print("psi: ", psi_values)
print("w: ", w_values)
print("total_time: ", total_time)

1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
5 6
5 7
5 8
5 9
5 10
5 11
5 12
6 7
6 8
6 9
6 10
6 11
6 12
7 8
7 9
7 10
7 11
7 12
8 9
8 10
8 11
8 12
9 10
9 11
9 12
10 11
10 12
11 12
w:  [ 0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0.]
phi:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
psi:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [